# Data Processing

## Pandas

Існує ряд бібліотек Python, які обробляють реляційні дані, зазвичай написані як інтерфейси до декількох різних систем управління реляційними базами даних (таких як PostgreSQL, MySQL та інші). [Примітка: слід зазначити, що програмне забезпечення, таке як PostreSQL і MySQL, правильніше називати системами управління реляційними базами даних (RDBMS), а не базами даних. База даних — це фактичні таблиці та записи, що визначають фактичну сукупність даних.

Однак у цьому курсі ми будемо працювати з реляційними даними переважно за допомогою двох бібліотек: Pandas і SQLite. Це особливо прості бібліотеки, якщо говорити про реальні бази даних: Pandas, безумовно, не є реальною системою реляційних баз даних (хоча вона надає функції, що віддзеркалюють деякі їхні можливості), тоді як SQLite є «реальною» RDBMS, але надзвичайно простою, без стандартної архітектури клієнт/сервер, яка є практично у всіх реальних виробничих базах даних. Проте для багатьох задач у галузі науки про дані вони будуть достатніми, тому ми зосередимося на них тут.

Ми вже коротко розглянули Panda, коли обговорювали збір даних, і вона виявилася однією з найкорисніших бібліотек Python для науки про дані. Як ми вже згадували вище (але будемо повторювати цей факт багато разів), Pandas — це не бібліотека реляційних баз даних, а бібліотека «даних-рам». Ви можете уявити собі фрейм даних як 2D-масив, за винятком того, що записи у фреймі даних можуть бути будь-яким типом об'єкта Python (і мати змішані типи в масиві), а рядки/стовпці можуть мати «мітки» замість цілочисельних індексів, як у стандартному масиві.

Давайте подивимося, як спочатку створити фрейм даних у Pandas, який відображає нашу таблицю Person вище (ми залишимо стовпець «Role ID» поза увагою, щоб спростити завдання).

In [1]:
import pandas as pd

df = pd.DataFrame([(1, 'Kolter', 'Zico'), 
                   (2, 'Xi', 'Edgar'),
                   (3, 'Lee', 'Mark'), 
                   (4, 'Mani', 'Shouvik'),
                   (5, 'Gates', 'Bill'),
                   (6, 'Musk', 'Elon')], 
                  columns=["id", "last_name", "first_name"])
df

,id,last_name,first_name
0,1,Kolter,Zico
1,2,Xi,Edgar
2,3,Lee,Mark
3,4,Mani,Shouvik
4,5,Gates,Bill
5,6,Musk,Elon


«Індекс» для Pandas насправді означає щось більше, ніж «первинний ключ» у таблиці бази даних (хоча з тим винятком, що можливі дублікати записів). 

Тобто індекс (якщо він зроблений правильно, без дублікатів) є ідентифікатором для кожного рядка в базі даних. Ми можемо встановити індекс для одного з існуючих стовпців за допомогою виклику `.set_index()`.

Але тут потрібно бути дуже обережним з одним моментом. За замовчуванням більшість операцій Pandas, таких як `.set_index()` та багато інших, не виконуються на місці. Тобто, хоча виклик df.set_index(«id») вище повертає копію фрейму даних df з індексом, встановленим для стовпця id (пам'ятайте, що Jupyter notebook відображає значення, яке повертається в останньому рядку комірки), оригінальний об'єкт df насправді тут не змінюється.

Якщо ми хочемо фактично змінити сам об'єкт df, потрібно використовувати прапор `inplace=True` для цих функцій (або присвоїти оригінальний об'єкт результату функції, але це не так чітко).

In [2]:
print(df.set_index("id"))

# df

df.set_index("id", inplace=True)
df


   last_name first_name
id                     
1     Kolter       Zico
2         Xi      Edgar
3        Lee       Mark
4       Mani    Shouvik
5      Gates       Bill
6       Musk       Elon


,last_name,first_name
id,,
1,Kolter,Zico
2,Xi,Edgar
3,Lee,Mark
4,Mani,Shouvik
5,Gates,Bill
6,Musk,Elon


In [8]:
# You can access individual elements using the `.loc[row, column]` notation, where row denotes the index you 
# are searching for and column denotes the column name.
df.loc[1, "last_name"]

# If we want to access all last names, (or all elements in a particular row), we use the : wildcard. For example
df.loc[:, "last_name"]

# We can pass a list of desired columns, to get a DataFRame return objet:
df.loc[:, ["last_name"]]

# We can do a similar thing with row indexes.
df.loc[[1,2],:]

# We can additionally use `.loc` to change the content of existing entries:
df.loc[1,"last_name"] = "Kilter"
df

# We can even add additional rows/columns
df.loc[7,:] = ('Moore', 'Andrew')
df

# Finally, remember that `.loc` always indexes based upon the “index” (i.e., effectively primary key) of the data 
# frame along with the column name. If you want to instead access based upon positional index (i.e., using 0-indexed 
# counters for both the rows and columns), you can use the `.iloc` property
df.iloc[4,1]

'Bill'

Методи очищення даних та обробка відсутніх значень

У реальному світі необроблені дані рідко бувають ідеальними. Вони часто містять помилки, невідповідності та відсутні значення. Очищення даних — це процес виявлення та виправлення цих проблем для поліпшення якості даних.

- *Дублікати*: це повторювані записи в наборі даних. Дублікати рядків можуть спотворити ваш аналіз або модель, особливо якщо вони представляють одну й ту саму подію або об'єкт.
- *Неправильні типи даних*: дані, які введені неправильно (наприклад, числа, збережені як рядки), можуть призвести до помилок в аналізі та навчанні моделі.
- *Випадкові значення*: екстремальні значення, які значно відрізняються від решти даних. Вони можуть спотворити результати статистичного аналізу або моделей машинного навчання.

Відсутність даних може відбуватися з різних причин, таких як помилки введення даних, несправності обладнання або проблеми з конфіденційністю. Важливо обережно поводитися з відсутніми значеннями, оскільки неправильне поводження може призвести до упереджених результатів.

Типи відсутніх даних:

- *MCAR* (Missing Completely at Random, повністю випадкова відсутність): ймовірність відсутності даних не залежить від самих даних.
- *MAR* (відсутність випадкова): ймовірність відсутності даних пов'язана з іншими спостережуваними даними, але не з самими відсутніми даними.
- *MNAR* (відсутність невипадкова): відсутність пов'язана з самими неспостережуваними даними.

**Видалення відсутніх даних**:

Видалення рядків або стовпців з відсутніми значеннями є найпростішим підходом, але може призвести до втрати цінної інформації.

In [1]:
import pandas as pd

df = pd.read_csv('../resources/data.csv')
print(df, '\n')
df_cleaned = df.dropna()  # Drops all rows with any missing values
# Drop rows with any missing values in numerical columns
df_feature_cleaned = df.dropna(subset=['feature1', 'feature2'])
print(df_cleaned, '\n')
print(df_feature_cleaned, '\n')

   feature1  feature2 categorical_column  target
0       1.2       3.2                  A      10
1       2.3       NaN                  B      15
2       3.1       1.1                  C       7
3       4.7       4.8                  A      18
4       NaN       2.7                  B      11
5       5.1       4.2                  C      19
6       2.8       NaN                  A      13
7       4.1       3.9                  B      16
8       5.3       4.1                NaN      14
9       3.3       2.1                  C      12 

   feature1  feature2 categorical_column  target
0       1.2       3.2                  A      10
2       3.1       1.1                  C       7
3       4.7       4.8                  A      18
5       5.1       4.2                  C      19
7       4.1       3.9                  B      16
9       3.3       2.1                  C      12 

   feature1  feature2 categorical_column  target
0       1.2       3.2                  A      10
2       3.1     

**Mean Imputation** (заповнення середнім): Замінити відсутні значення середнім значенням стовпця.

In [2]:
# Impute missing values with the mean of the respective column
df['feature1'].fillna(df['feature1'].mean(), inplace=True)
df['feature2'].fillna(df['feature2'].mean(), inplace=True)
df

,feature1,feature2,categorical_column,target
0,1.200000,3.2000,A,10
1,2.300000,3.2625,B,15
2,3.100000,1.1000,C,7
3,4.700000,4.8000,A,18
4,3.544444,2.7000,B,11
5,5.100000,4.2000,C,19
6,2.800000,3.2625,A,13
7,4.100000,3.9000,B,16
8,5.300000,4.1000,NaN,14
9,3.300000,2.1000,C,12


**Median Imputation**: Корисно, коли дані є викривленими.

In [3]:
df = pd.read_csv('../resources/data.csv')
df['feature1'].fillna(df['feature1'].median(), inplace=True)
df

,feature1,feature2,categorical_column,target
0,1.2,3.2,A,10
1,2.3,NaN,B,15
2,3.1,1.1,C,7
3,4.7,4.8,A,18
4,3.3,2.7,B,11
5,5.1,4.2,C,19
6,2.8,NaN,A,13
7,4.1,3.9,B,16
8,5.3,4.1,NaN,14
9,3.3,2.1,C,12


**Mode Imputation:** Часто використовується для категоріальних даних.

In [4]:
df['categorical_column'].fillna(df['categorical_column'].mode()[0], inplace=True)
df

,feature1,feature2,categorical_column,target
0,1.2,3.2,A,10
1,2.3,NaN,B,15
2,3.1,1.1,C,7
3,4.7,4.8,A,18
4,3.3,2.7,B,11
5,5.1,4.2,C,19
6,2.8,NaN,A,13
7,4.1,3.9,B,16
8,5.3,4.1,A,14
9,3.3,2.1,C,12


Розширене заповнення даних:

**Імпутування K-найближчих сусідів (KNN):** оцінює відсутні значення на основі значень найближчих сусідів.

**Багатовимірне імпутування:** враховує взаємозв'язки між ознаками для імпутування відсутніх значень.

**Створення індикаторних змінних:** інший підхід полягає у створенні бінарної індикаторної змінної, яка позначає наявність відсутніх даних.

## Масштабування та нормалізація даних (Data Scaling and Normalization)

Масштабування та нормалізація даних є важливими етапами попередньої обробки, які можуть суттєво вплинути на ефективність ваших моделей.

Масштабування даних гарантує, що всі ознаки однаково впливають на процес навчання моделі. Коли ознаки мають дуже різні діапазони, ті, що мають більші діапазони, можуть домінувати в обчисленнях відстані або кроках градієнта, що призводить до неоптимальних моделей.

*Алгоритми на основі відстані:* В алгоритмах, таких як K-найближчі сусіди (KNN) або машини опорних векторів (SVM), відстань між точками даних має вирішальне значення. Якщо одна ознака має набагато більший діапазон, ніж інші, вона непропорційно впливатиме на метрику відстані, що може спотворити результати.

*Градієнтний спуск:* Алгоритми, такі як лінійна регресія та нейронні мережі, використовують градієнтний спуск для мінімізації функції вартості. Якщо ознаки не масштабуються, алгоритм градієнтного спуску може потребувати більше часу для збіжності або може збігатися до неоптимального рішення через нерівномірні кроки, зроблені в просторі параметрів.

Мінімально-максимальне масштабування

Перемасштабовує дані до фіксованого діапазону, зазвичай [0, 1]. Ця техніка особливо корисна, коли ви знаєте, що ваші дані відповідають розподілу з чіткими верхньою та нижньою межами.

$$X' = \frac{X-X_{min}}{X_{max}-X_{min}}$$

- Використовуйте мінімально-максимальне масштабування, коли ви хочете, щоб усі ваші ознаки мали однаковий масштаб (наприклад, в алгоритмах, таких як KNN, SVM).
- Особливо корисно, коли дані розподілені по відомому і фіксованому діапазону.

In [5]:
from sklearn.preprocessing import MinMaxScaler
import pandas as pd

data = {'feature': [10, 50, 100, 150, 200]}
df = pd.DataFrame(data)

scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(df)

print(scaled_data)

[[0.        ]
 [0.21052632]
 [0.47368421]
 [0.73684211]
 [1.        ]]


In [6]:
df = pd.read_csv('../resources/data.csv')

# Select numerical columns for scaling
numerical_cols = df[['feature1', 'feature2']]

# Keep categorical and target columns separate
categorical_cols = df[['categorical_column', 'target']]

from sklearn.preprocessing import MinMaxScaler

# Initialize the scaler
scaler = MinMaxScaler()

# Fit and transform the numerical columns
numerical_scaled = scaler.fit_transform(numerical_cols)

# Convert the scaled data back to a DataFrame
numerical_scaled_df = pd.DataFrame(numerical_scaled, columns=numerical_cols.columns)

# Combine scaled numerical data with the categorical columns
df_scaled = pd.concat([numerical_scaled_df, categorical_cols.reset_index(drop=True)], axis=1)

# Display the final DataFrame
print(df_scaled)

   feature1  feature2 categorical_column  target
0  0.000000  0.567568                  A      10
1  0.268293       NaN                  B      15
2  0.463415  0.000000                  C       7
3  0.853659  1.000000                  A      18
4       NaN  0.432432                  B      11
5  0.951220  0.837838                  C      19
6  0.390244       NaN                  A      13
7  0.707317  0.756757                  B      16
8  1.000000  0.810811                NaN      14
9  0.512195  0.270270                  C      12


**Нормалізація за Z-балом (або стандартизація)**
**(Z-Score Normalization or standardization)**

Перемасштабовує дані так, щоб середнє значення дорівнювало 0, а стандартне відхилення — 1. Ця техніка корисна, коли дані мають гаусівський (нормальний) розподіл, але не обов'язково обмежені.

$$X' = \frac{X-\nu}{\sigma}$$

- Використовуйте нормалізацію за Z-балом, коли ваші дані мають нормальний розподіл.
- Це стандартний вибір для багатьох моделей машинного навчання, особливо тих, що передбачають нормальний розподіл даних.

In [7]:
from sklearn.preprocessing import StandardScaler
import pandas as pd

data = {'feature': [10, 50, 100, 150, 200]}
df = pd.DataFrame(data)

scaler = StandardScaler()
standardized_data = scaler.fit_transform(df)

print(standardized_data)

[[-1.35411306]
 [-0.76536825]
 [-0.02943724]
 [ 0.70649377]
 [ 1.44242478]]


**Надійне масштабування (Robust Scaling)**

Використовує медіану та міжквартильний розмах (IQR) для масштабування, що робить його надійним щодо винятків. На відміну від мінімально-максимального масштабування та нормалізації Z-балу, надійне масштабування менше піддається впливу екстремальних значень у даних.

$$X'=\frac{X - \text{медіана}(X)} {\text{IQR}(X)}$$

Де $\text{IQR}(X)$ — діапазон між 25-м процентилем ($Q1$) і 75-м процентилем ($Q3$).

- Використовуйте надійне масштабування, якщо ваші дані містять винятки або мають негаусівський розподіл.
- Ідеально підходить для ситуацій, коли дані є асиметричними або містять екстремальні значення, які не повинні сильно впливати на масштабування.

In [ ]:
from sklearn.preprocessing import RobustScaler
import pandas as pd

data = {'feature': [10, 50, 100, 150, 200]}
df = pd.DataFrame(data)

scaler = RobustScaler()
robust_scaled_data = scaler.fit_transform(df)

print(robust_scaled_data)

*Мінімально-максимальне масштабування*: використовується, коли відомі межі даних або коли алгоритм вимагає даних у певному діапазоні.

*Нормалізація за Z-балом*: підходить для нормально розподілених даних або коли потрібно стандартизувати ознаки, щоб вони мали однакову важливість.

*Надійне масштабування*: найкраще підходить для роботи з даними, що містять винятки.

## Вступ до конструювання ознак (Feature Engineering)

Інженерія ознак є важливим етапом у процесі попередньої обробки даних. Вона передбачає перетворення необроблених даних на ознаки, які краще відображають основну проблему для моделей прогнозування. Якісна інженерія ознак може значно покращити ефективність моделей машинного навчання, надаючи їм найбільш релевантні та інформативні вхідні дані.

*Інженерія ознак* — це процес відбору, модифікації або створення нових ознак з необроблених даних для покращення ефективності моделей машинного навчання. Метою є створення ознак, які ефективніше відображають основні закономірності в даних, ніж вихідні необроблені дані. Правильна інженерія ознак може призвести до спрощення моделей, підвищення точності та скорочення часу навчання.

**Створення нових ознак** 

Включає виведення нових змінних з існуючих, які краще відображають основну структуру даних. Це може включати математичні перетворення, агрегації або операції, специфічні для певної області.

In [8]:
import pandas as pd

data = {
    'height': [1.60, 1.75, 1.82, 1.90],
    'weight': [55, 80, 72, 90]
}
df = pd.DataFrame(data)

# Creating a new feature: BMI
df['BMI'] = df['weight'] / (df['height'] ** 2)

print(df)

   height  weight        BMI
0    1.60      55  21.484375
1    1.75      80  26.122449
2    1.82      72  21.736505
3    1.90      90  24.930748


**Кодування категоріальних змінних**

Алгоритми машинного навчання зазвичай вимагають числових даних. Тому категоріальні змінні необхідно перетворити в числовий формат. Існує кілька методів кодування категоріальних змінних:

Кодування *One-hot* перетворює кожну категорію в новий бінарний стовпець. Цей метод підходить для випадків, коли кількість категорій обмежена.

In [9]:
data = {'color': ['red', 'blue', 'green']}
df = pd.DataFrame(data)

# One-Hot Encoding
df_encoded = pd.get_dummies(df, columns=['color'])

print(df_encoded)

   color_blue  color_green  color_red
0           0            0          1
1           1            0          0
2           0            1          0


*Кодування міток* присвоює кожної категорії унікальне ціле число. Це простий метод, але він передбачає порядкові відносини між категоріями, що може бути недоречним у деяких випадках.

In [10]:
from sklearn.preprocessing import LabelEncoder

data = {'color': ['red', 'blue', 'green']}
df = pd.DataFrame(data)

# Label Encoding
encoder = LabelEncoder()
df['color_encoded'] = encoder.fit_transform(df['color'])

print(df)

   color  color_encoded
0    red              2
1   blue              0
2  green              1


*Цільове кодування* передбачає заміну кожної категорії середнім значенням цільової змінної для цієї категорії. Ця техніка є корисною, коли категоріальна змінна має велику кількість рівнів.

In [11]:
data = {
    'color': ['red', 'blue', 'green', 'red', 'blue', 'green'],
    'target': [1, 0, 0, 1, 1, 0]
}
df = pd.DataFrame(data)

# Target Encoding
df['color_encoded'] = df.groupby('color')['target'].transform('mean')

print(df)

   color  target  color_encoded
0    red       1            1.0
1   blue       0            0.5
2  green       0            0.0
3    red       1            1.0
4   blue       1            0.5
5  green       0            0.0


**Особливості взаємодії**

Особливості взаємодії створюються шляхом поєднання двох або більше змінних для відображення взаємодії між ними. Ці особливості можуть виявити взаємозв'язки, які не є очевидними при розгляді змінних окремо.

*Приклад*:

Припустимо, що ви маєте такі даны, як вік і дохід користувачів. Термін взаємодії можна створити шляхом множення цих двох властивостей, щоб відобразити, як дохід змінюється з віком.

In [12]:
data = {
    'age': [25, 35, 45, 55],
    'income': [30000, 50000, 70000, 90000]
}
df = pd.DataFrame(data)

# Creating an interaction feature
df['age_income_interaction'] = df['age'] * df['income']

print(df)

   age  income  age_income_interaction
0   25   30000                  750000
1   35   50000                 1750000
2   45   70000                 3150000
3   55   90000                 4950000


**Поліноміальні ознаки**

Створюються шляхом генерації нових ознак, які є поліноміальними комбінаціями існуючих ознак. Це може бути особливо корисно при моделюванні нелінійних взаємозв'язків.

*Приклад*:

Припустимо, у вас є одна ознака x. Ви можете створити поліноміальні ознаки, такі як x^2, x^3 тощо, які можуть допомогти у виявленні нелінійних взаємозв'язків у даних.

In [13]:
from sklearn.preprocessing import PolynomialFeatures

data = {'x': [2, 3, 4]}
df = pd.DataFrame(data)

# Creating polynomial features (degree 2)
poly = PolynomialFeatures(degree=2, include_bias=False)
poly_features = poly.fit_transform(df)

# Convert the polynomial features back to a DataFrame
df_poly = pd.DataFrame(poly_features, columns=poly.get_feature_names_out(['x']))

df_poly

,x,x^2
0,2.0,4.0
1,3.0,9.0
2,4.0,16.0
